<a href="https://colab.research.google.com/github/nonmetal/ssodam-qa-generator/blob/main/ssodam_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
import time

In [ ]:
# configuration
max_elements = 100  # max number of elements to print for each category
timedifference = 8 * 60  # (ssodam time zone = GMT+9) - (crawler time zone = GMT+1) in minutes
first_page = 1  # delete this line to crawl from the beginning of the board
last_page = 848  # delete this line to crawl until the end of the board
main_url = "http://www.ssodam.com/"
login_url = "http://www.ssodam.com/auth"
board_url_base = "http://www.ssodam.com/board/4/"  # board url without page number
content_url_base = "http://www.ssodam.com/content/"  # content url without post number

## Original Codes
* Class Post
* Board Crawl Function
* Statistics

In [ ]:
class Post:
    def __init__(self, feed, title="", comments=0, likes=0, postdate=date(1, 1, 1), views=0):
        if isinstance(feed, str):
            (postid, title, comments, likes, postdate, views) = feed.strip().split("\t")
            (year, month, day) = postdate.split("-")
            postdate = date(int(year), int(month), int(day))
        else:
            postid = feed if isinstance(feed, int) else 0

        self.postid = int(postid)
        self.title = title
        self.comments = int(comments)
        self.likes = int(likes)
        self.postdate = postdate
        self.views = int(views)

    def __str__(self):
        try:
            return "%d\t%s\t%d\t%d\t%s\t%d" % (
                self.postid, self.title, self.comments,
                self.likes, self.postdate, self.views)
        except UnicodeDecodeError:
            return "%d\t%s\t%d\t%d\t%s\t%d" % (
                self.postid, self.title, self.comments,
                self.likes, self.postdate, self.views)

In [ ]:
def crawl():
    session = requests.session()

# Login
    print("Login Attempt")
    #username, password = input("\tid: "), input("\tpassword: ")
    csrftoken = session.get(main_url).cookies["csrftoken"]
    payload = {
        "id": 'jyjoon',
        "password": 'p970801k',
        "auto": "false",
        "csrfmiddlewaretoken": csrftoken
    }
    result = session.post(login_url, data=payload, headers={"Referer": main_url})
    if result.status_code != 200:
        print("Login Failure")
        return
    else:
        print("Login Success")

# Get first_ and last_page
    result = session.get(board_url_base + "1")
    soup = BeautifulSoup(result.text, "lxml")
    #if "first_page" not in globals():
    #    first_page = 1
    #if "last_page" not in globals():
    #    last_page = int(soup.text[soup.text.find("max_page"):].split('\n')[0][0:-1].split(" ")[1])

    # figure out announcements
    announcements = [911221, 1]  # contains post number of announcements
    #for label in soup.find_all("span", "label label-info"):
    #    announcements.add(int(label.find_next_sibling("a")['href'].split('/')[2].split('?')[0]))

    year = (datetime.now() + timedelta(minutes=timedifference)).year

    # crawl each page and collect data
    f_posts = open("/content/posts.txt", "w")
    posts = []
    print("Crawl Start")
    for page in range(first_page, last_page + 1):
        board_url = board_url_base + str(page)
        result = session.get(board_url)
        soup = BeautifulSoup(result.text, "lxml")
        for td in soup.find_all("td", "mobile-hide"):
            try:
                if int(td.text) in announcements:
                    continue
            except ValueError:
                continue
            postid = int(td.text)
            td = td.find_next("td", "title title-align")
            if td.find("a", href="alert();") is not None:
                continue  # Frozen post
            title = td.find("a").text.strip()
            try:
                comments = int(td.find("span", "comment-num").text.strip("[]"))
            except (AttributeError, ValueError):  # post with no comments
                comments = 0
            td = td.find_next("td").find_next("td")
            likes = int(td.text.strip())
            td = td.find_next("td")
            try:
                month, day = [int(x) for x in td.text.strip().split('/')]
                if (month, day) == (12, 31) and posts[-1].postdate.month == 1:
                    year -= 1
            except (ValueError, IndexError):
                continue  # too recent post, not 24 hours old yet
            postdate = date(year, month, day)
            td = td.find_next("td")
            views = int(td.text.strip())
            posts.append(Post(postid, title, comments, likes, postdate, views))
            f_posts.write(str(posts[-1]) + "\n")
        print("\tReading page %d of %d (%.2f%%)" % (
            page, last_page, 100.0 * (page - first_page + 1) / (last_page - first_page + 1)))
        time.sleep(1)
    print("Crawl Done")
    f_posts.close()
    session.close()
    return posts

In [ ]:
def statistics(posts):
    global max_elements
    max_elements = min(max_elements, len(posts))

    print("Statistic Processing Start")
    print("\tComments")
    f_comments = open("/content/comments.txt", "w")
    most_comments = posts[:max_elements]
    most_comments.sort(key=lambda p: p.comments, reverse=True)
    for post in posts[max_elements:]:
        if post.likes > most_comments[-1].comments:
            most_comments[-1] = post
            most_comments.sort(key=lambda p: p.comments, reverse=True)
    for post in most_comments:
        f_comments.write(str(post) + "\n")
    f_comments.close()

    print("\tLikes")
    f_likes = open("/content/likes.txt", "w")
    most_likes = posts[:max_elements]
    most_likes.sort(key=lambda p: p.likes, reverse=True)
    for post in posts[max_elements:]:
        if post.likes > most_likes[-1].likes:
            most_likes[-1] = post
            most_likes.sort(key=lambda p: p.likes, reverse=True)
    for post in most_likes:
        f_likes.write(str(post) + "\n")
    f_likes.close()

    print("\tViews")
    f_views = open("/content/views.txt", "w")
    most_views = posts[:max_elements]
    most_views.sort(key=lambda p: p.views, reverse=True)
    for post in posts[max_elements:]:
        if post.views > most_views[-1].views:
            most_views[-1] = post
            most_views.sort(key=lambda p: p.views, reverse=True)
    for post in most_views:
        f_views.write(str(post) + "\n")
    f_views.close()

    print("\tDays")
    f_days = open("/content/days.txt", "w")
    count_by_weekday = {}
    count_by_date = {}
    for i in range(0, 7):
        count_by_weekday[i] = 0
    for post in posts:
        try:
            count_by_date[post.postdate] += 1
        except KeyError:
            count_by_date[post.postdate] = 1
        count_by_weekday[post.postdate.weekday()] += 1
    for weekday in count_by_weekday:
        f_days.write(
            "%s %d\n" % (["MON", "TUE", "WED", "THU", "FRI", "SAT", "SUN"][weekday], count_by_weekday[weekday]))
    f_days.write("\n")
    for postdate in count_by_date:
        f_days.write("%s %d\n" % (postdate, count_by_date[postdate]))
    f_days.close()

    print("Statistic Done")

In [ ]:
def readposts():
    print("Read Posts Start")
    f_posts = open("/content/posts.txt", "w")
    posts = []
    for line in f_posts:
        posts.append(Post(line))
    f_posts.close()
    print("Read Posts Done")
    return posts

In [ ]:
def main():
    posts = crawl()
    if posts is None:
        return
    statistics(posts)

In [ ]:
if __name__ == "__main__":
    main()

Login Attempt
Login Success
Crawl Start
	Reading page 1 of 848 (0.12%)
	Reading page 2 of 848 (0.24%)
	Reading page 3 of 848 (0.35%)
	Reading page 4 of 848 (0.47%)
	Reading page 5 of 848 (0.59%)
	Reading page 6 of 848 (0.71%)
	Reading page 7 of 848 (0.83%)
	Reading page 8 of 848 (0.94%)
	Reading page 9 of 848 (1.06%)
	Reading page 10 of 848 (1.18%)
	Reading page 11 of 848 (1.30%)
	Reading page 12 of 848 (1.42%)
	Reading page 13 of 848 (1.53%)
	Reading page 14 of 848 (1.65%)
	Reading page 15 of 848 (1.77%)
	Reading page 16 of 848 (1.89%)
	Reading page 17 of 848 (2.00%)
	Reading page 18 of 848 (2.12%)
	Reading page 19 of 848 (2.24%)
	Reading page 20 of 848 (2.36%)
	Reading page 21 of 848 (2.48%)
	Reading page 22 of 848 (2.59%)
	Reading page 23 of 848 (2.71%)
	Reading page 24 of 848 (2.83%)
	Reading page 25 of 848 (2.95%)
	Reading page 26 of 848 (3.07%)
	Reading page 27 of 848 (3.18%)
	Reading page 28 of 848 (3.30%)
	Reading page 29 of 848 (3.42%)
	Reading page 30 of 848 (3.54%)
	Reading 

In [ ]:
from google.colab import files
files.download("posts.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## New Codes
* Class Contents
* Contents Crawl Function


In [ ]:
def crawl2():
    session = requests.session()

# Login
    print("Login Attempt")
    #username, password = input("\tid: "), input("\tpassword: ")
    csrftoken = session.get(main_url).cookies["csrftoken"]
    payload = {
        "id": 'jyjoon',
        "password": 'p970801k',
        "auto": "false",
        "csrfmiddlewaretoken": csrftoken
    }
    result = session.post(login_url, data=payload, headers={"Referer": main_url})
    if result.status_code != 200:
        print("Login Failure")
        return
    else:
        print("Login Success")

# Figure out announcements
    announcements = "911221"

    # crawl each page and collect data
    f_posts = open("/content/posts.txt", "w")
    posts = []
    post_number = [911221, 1268653, 1267713, 1267557]
    print("Crawl Start")
    for page in post_number:
        postid = page
        content_url = content_url_base + str(page)
        result = session.get(content_url)
        soup = BeautifulSoup(result.text, "lxml")
        title = soup.find_all("div", "board-title")[0].text
        for td in soup.find_all("div", "mverdana comment-content"):
            try:
                comments = []
                temp = td.text
                comments.append(temp)
            except (ValueError, IndexError):
                continue
            posts.append(Contents(postid, title, comments[0]))
            print(str(posts[-1])
            print(str(posts))
            f_posts.write(str(posts[-1]) + "\n")
        print("\tReading page %d of %d (%.2f%%)" % (
            page, len(post_number), 100.0 * (page - 1) / (len(post_number) - 1)))

    print("Crawl Done")
    f_posts.close()
    session.close()
    return posts

In [ ]:
class Contents:
    def __init__(self, feed, title="", comments=""):
        if isinstance(feed, str):
            (postid, title, comments) = feed.strip().split("\t")
        else:
            postid = feed if isinstance(feed, int) else 0

        self.postid = int(postid)
        self.title = title
        self.comments = comments

    def __str__(self):
        try:
            return "%d\t%s\t%s" % (
                self.postid, self.title, self.comments)
        except UnicodeDecodeError:
            return "%d\t%s\t%s" % (
                self.postid, self.title, self.comments)

In [ ]:
import chardet

s = b'\xeb\x84\xa4\xec\x9d\xb4\xeb\xb2\x84'
type(s), s
u = s.decode('utf-8')
type(u), u

(str, '네이버')